In [1]:
#%pip install langchain
#%pip install "langserve[all]"
#%pip install langchain-cli
#%pip install langchain-openai
#%pip install python-dotenv
#%pip install openai
#%pip install unstructured
#%pip install faiss-cpu
#%pip install chromadb
#%pip install langchainhub
#%pip install pypdf

In [2]:
import openai_setup

AZURE_OPENAI_ENDPOINT:https://anildwaopenaiwestus.openai.azure.com/


In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/PayPal SDD - Migration Example no JSDK.pdf")
pdf_pages = loader.load_and_split()

from langchain_community.document_loaders import UnstructuredURLLoader 

urls  = ["https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/",
         "https://developer.paypal.com/api/nvp-soap/get-started/",
         "https://developer.paypal.com/api/nvp-soap/PayPalSOAPAPIArchitecture/",
         "https://developer.paypal.com/api/nvp-soap/apiCredentials/",
         "https://developer.paypal.com/api/nvp-soap/nvpsoap-sdks/",
         "https://developer.paypal.com/api/nvp-soap/set-express-checkout-nvp/",
         "https://developer.paypal.com/api/nvp-soap/set-express-checkout-soap/",
         "https://developer.paypal.com/api/nvp-soap/endpoints/",
         ]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

page_contents = []
for d in data:
    
    print(d.metadata)
    texts = text_splitter.create_documents([d.page_content])
    for text in texts:
        text.metadata = d.metadata
        page_contents.append(text)
    #page_contents.append(text.page_content for text in texts)

for text in page_contents:
    print(text.metadata)


for p in pdf_pages:
    print(p.metadata)
    texts = text_splitter.create_documents([p.page_content])
    for text in texts:
        text.metadata = d.metadata
        page_contents.append(text)

Created a chunk of size 1431, which is longer than the specified 1000
Created a chunk of size 1172, which is longer than the specified 1000
Created a chunk of size 1232, which is longer than the specified 1000
Created a chunk of size 8055, which is longer than the specified 1000
Created a chunk of size 1324, which is longer than the specified 1000
Created a chunk of size 9014, which is longer than the specified 1000
Created a chunk of size 6614, which is longer than the specified 1000
Created a chunk of size 1732, which is longer than the specified 1000
Created a chunk of size 1032, which is longer than the specified 1000
Created a chunk of size 2232, which is longer than the specified 1000
Created a chunk of size 5449, which is longer than the specified 1000
Created a chunk of size 8152, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 8456, which is longer than the specified 1000
Created a chunk of s

{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/get-started/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/PayPalSOAPAPIArchitecture/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/apiCredentials/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/nvpsoap-sdks/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/set-express-checkout-nvp/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/set-express-checkout-soap/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/endpoints/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/api/nvp-soap/NVPAPIOverview/'}
{'source': 'https://developer.paypal.com/a

In [5]:
import os
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings


embeddings_from_model = AzureOpenAIEmbeddings(azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"])
#embeddings = embeddings_model.embed_documents(page_contents)

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_from_model, store, namespace=embeddings_from_model.model
)

In [6]:
list(store.yield_keys())

['text-embedding-ada-00200b2e0d7-5fdd-5d09-b9be-8633095f947a',
 'text-embedding-ada-00202e25cdf-dfeb-578d-b5fa-b642d93570a6',
 'text-embedding-ada-00202ecf434-51d5-567a-9232-03dfd06a5389',
 'text-embedding-ada-0020708f7e8-80c8-5f3f-aeab-9744403c2acb',
 'text-embedding-ada-00207c6ba5b-3094-5407-8a17-370af896bb21',
 'text-embedding-ada-0020fb2c768-6de3-5600-aebb-37231e7e1829',
 'text-embedding-ada-002100db211-312b-5c1f-8fb2-15992c726b49',
 'text-embedding-ada-00211c9d6dc-aa88-5c68-a597-f94770a434a3',
 'text-embedding-ada-002125c0768-2d23-531c-a538-65e9d0cbab92',
 'text-embedding-ada-00213e35ab0-f343-5f3d-ae24-341614751073',
 'text-embedding-ada-002153773db-41f7-5fd8-9199-79ea309bdc61',
 'text-embedding-ada-00215e33b79-4ae9-574e-a00b-6b9857d835bb',
 'text-embedding-ada-00215f47f6b-88c6-59a5-92ff-5256142a4df2',
 'text-embedding-ada-0021792e946-23a6-5cc2-848a-1e7cb6796af9',
 'text-embedding-ada-0021796e6d0-6696-5389-8406-9d7079f1fe52',
 'text-embedding-ada-0021999899f-82fd-5086-be1c-303f6df

In [7]:
%%time
try: 
    faiss_db = FAISS.load_local("faiss_index", cached_embedder)
except(Exception) as e:
    print("Local index not found, creating new one")   
    faiss_db = FAISS.from_documents(page_contents, cached_embedder)
    faiss_db.save_local("faiss_index")

CPU times: total: 15.6 ms
Wall time: 515 ms


In [8]:
len(list(store.yield_keys()))

126

In [9]:
from langchain_community.vectorstores import Chroma
chroma_db = Chroma.from_documents(page_contents, AzureOpenAIEmbeddings(), persist_directory="./chroma")

In [10]:
query = "how to use paypal api as client server model"

embedding_vector = AzureOpenAIEmbeddings().embed_query(query)
docs = chroma_db.similarity_search_by_vector(embedding_vector, k=5)
print(docs[0].page_content)

Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.

PayPal API client-server architecture

The PayPal API uses a client-server model. Your website is a client of the PayPal server.

A page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.


In [11]:
faiss_retriever = faiss_db.as_retriever()
docs = faiss_retriever.get_relevant_documents(query)
print(docs[0].page_content)

Note: The PayFlow API also uses name-value pairs to provide parameter-based association between request and response fields of a message and their values; however, the PayFlow API is not the same as the NVP API; for more information about the PayFlow API, see the Payflow Gateway Developer Guide and Reference.

PayPal API client-server architecture

The PayPal API uses a client-server model. Your website is a client of the PayPal server.

A page on your website initiates an action on a PayPal API server by sending a request to the server. The PayPal server responds with a confirmation that the requested action was taken or indicates that an error occurred. The response might also contain additional information related to the request. The following diagram shows the basic request-response mechanism.
